# Step 1: To install the underlying packages

In [1]:
import micropip
await micropip.install('https://jupyterlite.readthedocs.io/multidict-4.7.6-py3-none-any.whl', keep_going=True)
await micropip.install('https://jupyterlite.readthedocs.io/frozenlist-1.4.0-py3-none-any.whl', keep_going=True)

In [3]:
await micropip.install('https://jupyterlite.readthedocs.io/urllib3-2.1.0-py3-none-any.whl', keep_going=True)
await micropip.install('https://jupyterlite.readthedocs.io/aiohttp-3.9.1-py3-none-any.whl', keep_going=True)

<class 'zipfile.BadZipFile'>: File is not a zip file

In [2]:
await micropip.install('https://jupyterlite.readthedocs.io/openai-1.6.1-py3-none-any.whl', keep_going=True)

<class 'zipfile.BadZipFile'>: File is not a zip file

In [ ]:
await micropip.install("ssl")
import ssl
await micropip.install("httpx", keep_going=True)
import httpx
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import json

# Step 2: To prepare a special http_client for your OpenAI connection

In [ ]:
import httpx
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class URLLib3Transport(httpx.BaseTransport):
    def __init__(self):
        self.pool = urllib3.PoolManager()

    def handle_request(self, request: httpx.Request):
        payload = json.loads(request.content.decode("utf-8").replace("'",'"'))
        urllib3_response = self.pool.request(request.method, str(request.url), headers=request.headers, json=payload)
        content = json.loads(urllib3_response.data.decode('utf-8'))
        stream = httpx.ByteStream(json.dumps(content).encode("utf-8"))
        headers = [(b"content-type", b"application/json")]
        return httpx.Response(200, headers=headers, stream=stream)

client = httpx.Client(transport=URLLib3Transport())


# Step 3: To call OpenAI API as usual with your created http_client, which was introduced since OpenAI Python version x.x.x

## (Please use an Azure service instead.)

In [ ]:
import json
from openai import OpenAI

openai_client = OpenAI(
    base_url="https://api.openai.com/v1/",
    api_key="your key here",
    http_client=client # call the transport client here
)
response = openai_client.chat.completions.with_raw_response.create(
    messages=[{
        "role": "user",
        "content": "sing me a song",
    }],
    model="gpt-3.5-turbo",
    max_tokens=30,
    temperature=0.7
)
completion = response.parse()
print(completion)


ChatCompletion(id='chatcmpl-8UnVuARz8PKRtoR1XBAKmADYvTRcj', choices=[Choice(finish_reason='length', index=0, message=ChatCompletionMessage(content="Sure! Here's a short song for you:\n\n(Verse 1)\nIn a world of endless dreams,\nWhere the sun forever beams,\nLet me", role='assistant', function_call=None, tool_calls=None))], created=1702350862, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=30, prompt_tokens=11, total_tokens=41))


In [ ]:
print(completion.choices[0].message.content)

Sure! Here's a short song for you:

(Verse 1)
In a world of endless dreams,
Where the sun forever beams,
Let me
